# Import libraries

Import required libraries and initialize Stockfish engine

In [ ]:
import chess
import chess.engine
import chess.pgn
import os
import math

In [ ]:
stockfish = chess.engine.SimpleEngine.popen_uci(os.path.join('bin', 'stockfish_14_x64'))
maia = chess.engine.SimpleEngine.popen_uci(os.path.join(os.path.expanduser('~'), 'repos', 'lc0', 'build', 'release', 'lc0'))

# Load pgns

In [ ]:
def games(pgn):
    while game := chess.pgn.read_game(pgn):
        yield game

In [ ]:
lichess_2013 = open(os.path.join('data', 'lichess_db_standard_rated_2013-01.pgn'))

# Define pipeline functions

* Calculate value of a piece
* Get Stockfish evaluations of tactic move suggestions
* Get top-n Stockfish move suggestions for a given board position
* Evaluate the move suggestions from a tactic against Stockfish suggestions using DCG (discounted
  cumulative gain)


In [ ]:
def value(piece_type):
    king_values = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 99
    }
    return king_values[piece_type]

In [ ]:
ray_piece_types = [chess.QUEEN, chess.ROOK, chess.BISHOP]

def is_defended(board, piece, square) -> bool:
    if board.attackers(piece.color, square):
        return True
    # ray defense https://lichess.org/editor/6k1/3q1pbp/2b1p1p1/1BPp4/rp1PnP2/4PRNP/4Q1P1/4B1K1_w_-_-_0_1
    for attacker in board.attackers(not piece.color, square):
        attacker_piece = board.piece_at(attacker)
        assert(attacker_piece)
        if attacker_piece.piece_type in ray_piece_types:
            bc = board.copy(stack = False)
            bc.remove_piece_at(attacker)
            if bc.attackers(piece.color, square):
                return True

    return False

def is_hanging(board, piece, square) -> bool:
    return not is_defended(board, piece, square)


In [ ]:
def behind(a, b):
    whole_ray = chess.SquareSet.ray(a, b)
    b_first = False
    for square in whole_ray:
        if square == b:
            b_first = True
            break
        if square == a:
            b_first = False
            break
    if b_first:
        for square in reversed(list(whole_ray)):
            whole_ray.discard(square)
            if square == b:
                break
        partial_ray = reversed(list(whole_ray))
    else:
        for square in whole_ray:
            whole_ray.discard(square)
            if square == b:
                break
        partial_ray = whole_ray
    return list(partial_ray)

In [ ]:
def get_evals(engine, board, suggestions):
    evals = []
    analysis = engine.analyse(board, limit=chess.engine.Limit(time=1), multipv=len(suggestions), root_moves=suggestions)
    # print(analysis)
    evals = [(root['score'].relative, root['pv'][0]) for root in analysis]
    return evals

In [ ]:
def get_top_n_moves(engine, n, board):
    analysis = engine.analyse(board, limit=chess.engine.Limit(time=1), multipv=n)
    # print(analysis)
    top_n_moves = [(root['score'].relative, root['pv'][0]) for root in analysis]
    return top_n_moves

In [ ]:
def evaluate(evaluated_suggestions, top_moves):
    dcg = 0
    for idx, (evaluated_move, top_move) in enumerate(zip(evaluated_suggestions, top_moves)):
        # print(evaluated_move, top_move)
        score, move = evaluated_move
        eval = score.score(mate_score=2000)
        score_top, move_top = top_move
        top_eval = score_top.score(mate_score=2000)
        dcg += abs(top_eval - eval) / math.log2(1 + (idx + 1))
    return dcg

# Define PAL pattern heuristics

Tactics implemented -

1. Can threat
2. Can fork
3. Can check
4. Discovered check
5. Discovered threat
6. Skewer
7. Pin

## In Check

An opponent's piece has a plausible move to the place of the King.

```
in_check(Side,KPlace,OPiece,OPlace,Pos) :-
    contents(Side,king,KPlace,Pos),
    contents(OSide,OPiece,OPlace,Pos),
    other_side(Side,OSide),
    piece_move(OSide,OPiece,OPlace,KPlace,Pos).
```

In [ ]:
def in_check(board):
    return board.is_check()

## Legal Move

Defines legal moves of chess pieces. A piece _Piece_ can move
to place _Place_ if it does not create a check on its own King.

```prolog
legal_move(Side,Piece,Place,NewPlace,Pos1) :-
    contents(Side,Piece,Place,Pos1),
    piece_move(Side,Piece,Place,NewPlace,Pos1),
    do_move(Side,Piece,Place,NewPlace,Pos1,Pos2),
    \+ in_check(Side,_,_,_,Pos2).
```

In [ ]:
def legal_move(board, move):
    return move in board.legal_moves

## Stale

A piece cannot move and the opponent's King is not in check.

```prolog
stale(Side,Piece,Place,Pos) :-
    contents(Side,Piece,Place,Pos),
    \+ legal_move(Side,Piece,Place,_,Pos),
    other_side(Side,OSide),
    \+ in_check(OSide,_,_,_,Pos).
```

In [ ]:
def stale(board, square):
    for move in board.legal_moves:
        if move.from_square == square:
            return False
    return True

## Threat1 (static)

One piece (P1) threatens an opponent's piece (P2) if there is a legal move of P1 to the place of
P2. The difference between _threat_ and _threat1_ is that the former is applicable only when the 
opponent's side is not in check.

```prolog
threat1(S1,P1,(X1,Y1),S2,P2,(X2,Y2),Pos) :-
    contents(S2,P2,(X2,Y2),Pos),
    other_side(S2,S1),
    legal_move(S1,P1,(X1,Y1),(X2,Y2),Pos).
```

In [ ]:
def threat1(board):
    pass

## Threat

A piece (P1) threatens an opponent's piece (P2) if P1 can capture P2.

```prolog
threat(S1,P1,(X1,Y1),S2,P2,(X2,Y2),Pos1) :-
    contents(S2,P2,(X2,Y2),Pos1),
    other_side(S2,S1),
    make_move(S1,P1,(X1,Y1),(X2,Y2),Pos1,Pos2),
    \+ contents(S2,P2,(X2,Y2),Pos2).
```

In [ ]:
def threat(board):
    pass

## Fork

There is a "special" fork if a piece (P3) threatens another piece (P2) and checks the King at the
same time.

```prolog
fork(S1,king,(X1,X2),S1,P2,(X2,Y2),S2,P3,(X3,Y3),Pos) :-
    other_side(S2,S1),
    in_check(S1,(X1,Y1),P3,(X3,Y3),Pos),
    legal_move(S2,P3,(X3,Y3),(X2,Y2),Pos).
```

```prolog
fork(S1,king,(X1,X2),S1,P2,(X2,Y2),S2,P3,(X3,Y3),Pos) :-
    in_check(S1,(X1,Y1),P3,(X3,Y3),Pos),
    threat1(S2,P3,(X3,Y3),S1,P2,(X2,Y2),Pos).
```

In [ ]:
def fork(board):
    pass

## Can Threat

A piece (P1) can threaten another piece (P2) after making a move to (X3,Y3).

```prolog
can_threat(S1,P1,(X1,Y1),S2,P2,(X2,Y2),(X3,Y3),Pos1) :-
    contents(S1,P1,(X1,Y1),Pos1),
    \+ threat(S1,P1,(X1,Y1),S2,P2,(X2,Y2),Pos1),
    make_move(S1,P1,(X1,Y1),(X3,Y3),Pos1,Pos2),
    threat(S1,P1,(X3,Y3),S2,P2,(X2,Y2),Pos2).
```

In [ ]:
def can_threat(board, limit=3):
    match = False
    suggestions = []
    for move in board.legal_moves:
        board.push(move)
        board.push(chess.Move.null())
        nb = 0
        # print(move, chess.square_name(move.to_square), board.attacks(move.to_square))
        for attacks in board.attacks(move.to_square):
            attacked_piece = board.piece_at(attacks)
            # print(chess.square_name(attacks), attacked_piece, value(attacked_piece))
            if attacked_piece is None or attacked_piece.color == board.turn:
                continue
            nb += 1
        board.pop()
        board.pop()
        if nb > 0:
            match = True
            suggestions.append(move)
    suggestions = list(set(suggestions))
    return match, suggestions[:limit]

## Can Fork

A piece (P1) can produce a fork to the opponent's King and piece (P3) after making a move to
(X4,Y4).

```prolog
can_fork(S1,P1,(X1,Y1),S2,king,(X2,Y2),S2,P3,(X3,Y3),(X4,Y4),Pos1)  
    contents(S1,P1,(X1,Y1),Pos),
    \+ fork(S2,king,(X2,Y2),S2,P3,(X3,Y3),S1,P1,(X1,Y1),Pos1),
    make_move(S1,P1,(X1,Y1),(X4,Y4),Pos1,Pos2),
    fork(S2,king,(X2,Y2),S2,P3,(X3,Y3),S1,P1,(X4,Y4),Pos2).
```

In [ ]:
def can_fork(board, limit=3):
    match = False
    suggestions = []
    for move in board.legal_moves:
        moved_piece_type = board.piece_type_at(move.from_square)
        if moved_piece_type == chess.KING:
            continue
        board.push(move)
        board.push(chess.Move.null())
        nb = 0
        king_attacked = False
        # print(move, chess.square_name(move.to_square), board.attacks(move.to_square))
        for attacks in board.attacks(move.to_square):
            attacked_piece = board.piece_at(attacks)
            # print(chess.square_name(attacks), attacked_piece, value(attacked_piece))
            if attacked_piece is None or attacked_piece.color == board.turn:
                continue
            if attacked_piece.piece_type == chess.KING:
                king_attacked = True
        board.pop()
        board.pop()
        if nb > 1 and king_attacked:
            match = True
            suggestions.append(move)
    suggestions = list(set(suggestions))
    return match, suggestions[:limit]

## Can Check

A piece (P1) can check the opponent's King after a moving to (X3,Y3).

```prolog
can_check(S1,P1,(X1,Y1),S2,king,(X2,Y2),(X3,Y3),Pos1) :-
    contents(S1,P1,(X1,Y1),Pos1),
    other_side(S1,S2),
    \+ in_check(S2,(X2,Y2),P1,(X1,Y1),Pos1),
    make_move(S1,P1,(X1,Y1),(X3,Y3),Pos1,Pos2),
    in_check(S2,(X2,Y2),P2,(X3,Y3),Pos2).
```

In [ ]:
def can_check(board, limit=3):
    match = False
    suggestions = []
    for move in board.legal_moves:
        if board.gives_check(move):
            match = True
            suggestions.append(move)
    suggestions = list(set(suggestions))
    return match, suggestions[:limit]

## Discovered Check

A check by piece (P2) can be "discovered" after moving another piece (P1) to (X4,Y4).

```prolog
disc_check(S1,P1,(X1,Y1),S1,P2,(X2,Y2),S2,king,(X3,Y3),(X4,Y4),Pos1) :-  
    contents(S1,P1,(X1,Y1),Pos1),
    other_side(S1,S2),
    sliding_piece(P1,(X1,Y1),Pos1),
    \+ in_check(S2,(X3,Y3),P1,(X1,Y1),Pos1),
    make_move(S1,P2,(X2,Y2),(X4,Y4),Pos1,Pos2),
    in_check(S2,(X3,Y3),P1,(X1,Y1),Pos2).
```

In [ ]:
def discovered_check(board, limit=3):
    match = False
    suggestions = []
    # print(board)
    # print(board.legal_moves)
    for move in board.legal_moves:
        # print(move)
        board.push(move)
        if not board.is_valid() or not board.is_check():
            board.pop()
            continue
        if move.to_square in board.checkers():
            board.pop()
            continue
        else:
            match = True
            suggestions.append(move)
        board.pop()
    suggestions = list(set(suggestions))
    return match, suggestions[:limit]

## Discovered Threat

A piece (P1) can
threat an opponent's piece (P3) after moving another piece (P2) to
(X4,Y4).

```prolog
disc_threat(S1,P1,(X1,Y1),S1,P2,(X2,Y2),S2,P3,(X3,Y3),(X4,Y4),Pos1) :- 
    sliding_piece(P1,(X1,Y1),Pos1),
    \+ threat(S1,P1,(X1,Y1),S2,P3,(X3,Y3),Pos1),
    make_move(S1,P2,(X2,Y2),(X4,Y4),Pos1,Pos2),
    threat(S1,P1,(X1,Y1),S2,P3,(X3,Y3),Pos2).
```

In [ ]:
def attacking_moves(board, side):
    "Finds the set of all moves for the input side which currently attack an opponent piece"
    result_set = set()
    for square in chess.SQUARES:
        curr_piece = board.piece_at(square)
        if curr_piece is not None and curr_piece.color == side:
            for attacked_square in board.attacks(square):
                attacked_piece = board.piece_at(attacked_square)
                if attacked_piece is not None and attacked_piece.color != side:
                    result_set.add(chess.Move(square, attacked_square))
    return result_set


In [ ]:
def discovered_threat(board, limit=3):
    match = False
    suggestions = []
    for move in board.legal_moves:
        prev_attacking_moves = attacking_moves(board, board.turn)
        board.push(move)
        board.push(chess.Move.null())
        curr_attacking_moves = attacking_moves(board, board.turn)
        move_diff = curr_attacking_moves - prev_attacking_moves
        if move_diff:
            match = True
            suggestions.append(move)
        board.pop()
        board.pop()
    suggestions = list(set(suggestions))
    return match, suggestions[:limit]

## Skewer

A King in check by a piece (P1)
"exposes" another piece (P3) when it is moved out of check to (X4,Y4).

```prolog
skewer(S1,P1,(X1,Y1),S2,king,(X2,Y2),S2,P3,(X3,Y3),(X4,Y4),Pos1) :- 
    sliding_piece(P1,(X1,Y1),Pos1),
    stale(S2,P3,(X3,Y3),Pos1),
    in_check(S2,(X2,Y2),P1,(X1,Y1),Pos1),
    \+ threat1(S1,P1,(X1,Y1),S2,P3,(X3,Y3),Pos1),
    make_move(S2,king,(X2,Y2),(X4,Y4),Pos1,Pos2),
    \+ stale(S2,P3,(X3,Y3),Pos2),
    \+ in_check(S2,(X2,Y2),P1,(X1,Y1),Pos2),
    threat1(S1,P1,(X1,Y1),S2,P3,(X3,Y3),Pos2).
```


In [ ]:
def skewer(board, limit=3):
    match = False
    suggestions = []
    # print(board)
    # print(board.legal_moves)
    for move in board.legal_moves:
        # print(move)
        moved_piece_type = board.piece_type_at(move.from_square)
        if moved_piece_type not in ray_piece_types:
            continue
        if not board.gives_check(move):
            continue
        board.push(move)
        board.push(chess.Move.null())
        if not board.is_valid():
            board.pop()
            board.pop()
            continue
        for attacked_square in board.attacks(move.to_square):
            attacked_piece = board.piece_at(attacked_square)
            if attacked_piece is None or attacked_piece.color == board.turn:
                continue
            # if piece immediately "behind" attacked piece is of opponent color and of lower value
            # than attacked piece, then match
            partial_ray = behind(move.to_square, attacked_square)
            # print(partial_ray)
            for square in partial_ray:
                if (xrayed_piece := board.piece_at(square)) is not None:
                    if (
                        xrayed_piece.color != board.turn and # opponent color
                        is_hanging(board, xrayed_piece, square) and # unsupported
                        value(xrayed_piece.piece_type) < value(attacked_piece.piece_type) # lesser value
                    ):
                        match = True
                        suggestions.append(move)
                    break
        board.pop()
        board.pop()
    suggestions = list(set(suggestions))
    return match, suggestions[:limit]

## Pin

A piece (P3) cannot move because it will produce a check on its
own side by piece (P1).

```prolog
pin1(S1,P1,(X1,Y1),S2,king,(X2,Y2),S2,P3,(X3,Y3),Pos1) :-
    sliding_piece(P1,(X1,Y1),Pos1),
    stale(S2,P3,(X3,Y3),Pos1),
    threat(S1,P1,(X1,Y1),S2,P3,(X3,Y3),Pos1),
    \+ in_check(S2,(X2,Y2),P1,(X1,Y1),Pos1),
    make_move(S1,P1,(X1,Y1),(X3,Y3),Pos1,Pos2),
    \+ contents(S2,P3,(X3,Y3),Pos2),
    \+ stale(S2,P3,(X3,Y3),Pos2),
    \+ threat(S1,P1,(X1,Y1),S2,P3,(X3,Y3),Pos2),
    in_check(S2,(X2,Y2),P1,(X3,Y3),Pos2).
```

This definition, however, is incorrect as it accepts positions where a
Pawn cannot move (i.e., stale) because it is "blocked" by another piece
rather than being unable to move because it creates a check on its own
King.

In [ ]:
def pin(board, limit=3):
    match = False
    suggestions = []
    # print(board)
    # print(board.legal_moves)
    for move in board.legal_moves:
        # print(move)
        moved_piece_type = board.piece_type_at(move.from_square)
        if moved_piece_type not in ray_piece_types:
            continue
        board.push(move)
        board.push(chess.Move.null())
        if not board.is_valid():
            board.pop()
            board.pop()
            continue
        for attacked_square in board.attacks(move.to_square):
            attacked_piece = board.piece_at(attacked_square)
            if (
                attacked_piece is None or # a piece is attacked
                attacked_piece.color == board.turn # attacked piece belongs to opponent
            ):
                continue
            # if piece immediately "behind" attacked piece is of opponent color, then match
            partial_ray = behind(move.to_square, attacked_square)
            # print(partial_ray)
            for square in partial_ray:
                if (xrayed_piece := board.piece_at(square)) is not None:
                    if (
                        xrayed_piece.color != board.turn # opponent color
                    ):
                        match = True
                        suggestions.append(move)
                    break
        board.pop()
        board.pop()
    suggestions = list(set(suggestions))
    return match, suggestions[:limit]

# Calculate metrics for all games

For each tactic -

1. coverage on lichess db (with position counts)
2. DCG for each engine (Maia-1100, Stockfish)

In [ ]:
total_games = 0  # total number of games
total_positions = 0 # total number of positions

engines = [stockfish, maia]
tactics = [can_threat, can_check, can_fork, discovered_check, discovered_threat, skewer, pin]
matches = {tactic.__name__: 0 for tactic in tactics}
dcgs = {tactic.__name__: {engine.id['name']: 0 for engine in engines} for tactic in tactics}

GAME_LIMIT = None
POS_LIMIT = None

for game in games(lichess_2013):
    print(f'Game [{total_games + 1}/?]')
    board = game.board()
    curr_positions = 0
    for move in game.mainline_moves():
        print(f'Move [{curr_positions + 1}/{len(game.mainline_moves())}]')
        board.push(move)
        for tactic in tactics:
            print(f'Current tactic: {tactic.__name__}')
            match, suggestions = tactic(board)
            if match:
                matches[tactic.__name__] += 1
                for engine in engines:
                    print(f'Current engine: {engine.id["name"]}')
                    evals = get_evals(engine, board, suggestions)
                    top_n_moves = get_top_n_moves(engine, len(suggestions), board)
                    # print(board.fen(), evaluated_suggestions, top_n_moves)
                    dcgs[tactic.__name__][engine.id['name']] += evaluate(evals, top_n_moves)
        curr_positions += 1
        total_positions += 1
        if POS_LIMIT and curr_positions >= POS_LIMIT:
            break
    total_games += 1
    if GAME_LIMIT and total_games >= GAME_LIMIT:
        break
            
print(f'# of games: {total_games}')
print(f'# of positions: {total_positions}')
print(f'Coverage: {matches}') # number of matched positions per game
print(f'DCG = {dcgs}')

In [ ]:
for engine in engines:
    engine.quit()

# Individual Test Position

A single constructed position for testing the constructed heuristics

In [ ]:
board = chess.Board('4k3/8/8/8/8/4B3/4R3/4K3 w - - 0 1')
board

In [ ]:
engine = maia
# print(engine.options)
print(engine.id['name'])

In [ ]:
match, suggestions = discovered_check(board)
print(suggestions)
analysis = engine.analyse(board, chess.engine.Limit(nodes=1), multipv=len(suggestions), root_moves=suggestions)
print(analysis)

In [ ]:
match, suggestions = discovered_check(board)
# print(match, suggestions)
top_moves = get_top_n_moves(engine, len(suggestions), board)
# print(top_moves)
if match:
    evals = get_evals(maia, board, suggestions)
    # print(board.fen(), match, list(zip(suggestions, evals)), top_moves)
print(evaluate(evals, top_moves))

In [ ]:
engine.quit()